In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# CrossProductComp

`CrossProductComp` performs a cross product between two 3-vector inputs.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    c_i = \bar{a}_i \times \bar{b}_i
$$

The first dimension of the inputs holds the vectorized dimension.
The default `vec_size` is 1, providing the cross product of $a$ and $b$ at a single
point.  The lengths of $a$ and $b$ at each point must be 3.

The shape of $a$ and $b$ will always be `(vec_size, 3)`, but the connection rules
of OpenMDAO allow the incoming connection to have shape `(3,)` when `vec_size` is 1, since
the storage order of the underlying data is the same.  The output vector `c` of
CrossProductComp will always have shape `(vec_size, 3)`.

## CrossProductComp Options

Options for CrossProductComp allow the user to rename the input variables $a$ and $b$ and the output $c$, as well as specifying their units.

In [2]:
import openmdao.api as om
om.show_options_table("openmdao.components.cross_product_comp.CrossProductComp")

Option,Default,Acceptable Values,Acceptable Types,Description
a_name,a,N/A,['str'],The variable name for vector a.
a_units,N/A,N/A,['str'],The units for vector a.
always_opt,False,"[True, False]",['bool'],"If True, force nonlinear operations on this component to be included in the optimization loop even if this component is not relevant to the design variables and responses."
b_name,b,N/A,['str'],The variable name for vector b.
b_units,N/A,N/A,['str'],The units for vector b.
c_name,c,N/A,['str'],The variable name for vector c.
c_units,N/A,N/A,['str'],The units for vector c.
distributed,False,"[True, False]",['bool'],"If True, set all variables in this component as distributed across multiple processes"
run_root_only,False,"[True, False]",['bool'],"If True, call compute, compute_partials, linearize, apply_linear, apply_nonlinear, and compute_jacvec_product only on rank 0 and broadcast the results to the other ranks."
vec_size,1,N/A,['int'],The number of points at which the cross product is computed


## CrossProductComp Constructor

The call signature for the `CrossProductComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.cross_product_comp.CrossProductComp.__init__
        :noindex:
```

## CrossProductComp Usage

There are often situations when numerous products need to be computed, essentially in parallel.
You can reduce the number of components required by having one `CrossProductComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The `add_product` method is used to create additional products after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.cross_product_comp.CrossProductComp.add_product
       :noindex:
```

## CrossProductComp Example

In the following example CrossProductComp is used to compute torque as the
cross product of force ($F$) and radius ($r$) at 100 points simultaneously.
Note the use of `a_name`, `b_name`, and `c_name` to assign names to the inputs and outputs.
Units are assigned using `a_units`, `b_units`, and `c_units`.
Note that no internal checks are performed to ensure that `c_units` are consistent
with `a_units` and `b_units`.


In [3]:
import numpy as np
import openmdao.api as om

n = 24

p = om.Problem()

p.model.add_subsystem(name='cross_prod_comp',
                      subsys=om.CrossProductComp(vec_size=n,
                                                 a_name='r', b_name='F', c_name='torque',
                                                 a_units='m', b_units='N', c_units='N*m'),
                      promotes_inputs=['r', 'F'])

p.setup()

p.set_val('r', np.random.rand(n, 3))
p.set_val('F', np.random.rand(n, 3))

p.run_model()

# Check the output in units of ft*lbf to ensure that our units work as expected.
expected = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    b_i = p.get_val('F')[i, :]
    expected.append(np.cross(a_i, b_i) * 0.73756215)

    actual_i = p.get_val('cross_prod_comp.torque', units='ft*lbf')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

print(p.get_val('cross_prod_comp.torque', units='ft*lbf'))

[[-1.18936444e-01  1.91054493e-01 -5.19382542e-02]
 [ 2.97876488e-01 -1.72063186e-01 -3.58983976e-01]
 [-9.41185579e-02 -5.26473693e-02  3.47970409e-01]
 [ 5.37691835e-02 -2.81972962e-01  2.00844812e-01]
 [-7.70879110e-02 -4.36756509e-02  6.94423571e-02]
 [ 6.18282640e-01 -8.65707363e-02 -4.15373710e-02]
 [-2.59604737e-01 -9.43734666e-02  2.43178453e-01]
 [-5.39404591e-01  4.15685330e-02  4.60579032e-01]
 [-2.07215393e-01  1.83469810e-01  5.03375167e-02]
 [-3.02773949e-01 -1.60637510e-01  3.50511373e-01]
 [-4.93704450e-02 -5.27817532e-02  9.47354579e-02]
 [-2.33430394e-01  1.88009396e-01 -6.21607638e-02]
 [-1.07803634e-01  1.95290768e-01 -5.00705408e-02]
 [-2.21730931e-01  5.44454796e-03  8.55911396e-02]
 [-4.12921956e-02  1.26554841e-03  4.48005306e-02]
 [-3.77086829e-02  5.27546802e-01 -1.36177414e-01]
 [ 4.91909797e-02 -4.85113914e-04 -1.05932599e-03]
 [ 1.90867819e-01 -1.90008903e-01 -1.84581380e-01]
 [-2.63876146e-02  4.91870310e-01 -3.84770085e-01]
 [ 2.57075883e-01 -5.79873351e-

In [4]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('cross_prod_comp.torque', units='ft*lbf'), np.array(expected), tolerance=1e-8)

2.0453786941082107e-09

## DotProductComp Example with Multiple Products

When defining multiple products:

- An input name in one call to `add_product` may not be an output name in another call, and vice-versa.
- The units and shape of variables used across multiple products must be the same in each one.

In [5]:
n = 24

p = om.Problem()

cpc = om.CrossProductComp(vec_size=n,
                          a_name='r', b_name='F', c_name='torque',
                          a_units='m', b_units='N', c_units='N*m')

cpc.add_product(vec_size=n,
                a_name='r', b_name='p', c_name='L',
                a_units='m', b_units='kg*m/s', c_units='kg*m**2/s')

p.model.add_subsystem(name='cross_prod_comp', subsys=cpc,
                      promotes_inputs=['r', 'F', 'p'])

p.setup()

p.set_val('r', np.random.rand(n, 3))
p.set_val('F', np.random.rand(n, 3))
p.set_val('p', np.random.rand(n, 3))

p.run_model()

# Check the output.
expected_T = []
expected_L = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    b_i = p.get_val('F')[i, :]
    expected_T.append(np.cross(a_i, b_i))

    actual_i = p.get_val('cross_prod_comp.torque')[i]
    rel_error = np.abs(expected_T[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

    b_i = p.get_val('p')[i, :]
    expected_L.append(np.cross(a_i, b_i))

    actual_i = p.get_val('cross_prod_comp.L')[i]
    rel_error = np.abs(expected_L[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

print(p.get_val('cross_prod_comp.torque'))

[[-0.06876089  0.08457726 -0.05763242]
 [-0.11373526  0.09530494  0.30703901]
 [ 0.36302229 -0.30412617  0.17736365]
 [ 0.15259094  0.11849427 -0.5383283 ]
 [ 0.093516   -0.07936575  0.06511162]
 [ 0.09787179 -0.18428999  0.11213479]
 [-0.12639718  0.09289152  0.02249633]
 [-0.65896564  0.72994393 -0.22402339]
 [ 0.22038835 -0.31337451  0.18760235]
 [-0.04723882 -0.05408839  0.48362206]
 [ 0.04572959  0.41048064 -0.15352234]
 [ 0.09090094  0.14962189 -0.24059373]
 [ 0.31218446 -0.02572959 -0.30455817]
 [-0.10607137  0.02769735  0.14588754]
 [-0.05806496  0.27051382 -0.23269651]
 [-0.16982266  0.22821032 -0.17088282]
 [-0.0577105   0.34137673  0.07458745]
 [-0.07839952  0.0467913   0.14572635]
 [-0.71647465  0.54117797  0.67021754]
 [-0.14212636 -0.15704736  0.22716696]
 [-0.00999064  0.24729715 -0.02494866]
 [ 0.57889361 -0.24243635 -0.34459383]
 [-0.02335254 -0.16654667  0.14787235]
 [-0.55425845 -0.06270983  0.57557943]]


In [6]:
print(p.get_val('cross_prod_comp.L'))

[[ 0.41874955  0.1240558  -0.25692693]
 [-0.01703562  0.03473554  0.04029163]
 [ 0.24797308 -0.19603267 -0.06487556]
 [ 0.4955497  -0.13312312 -0.6342708 ]
 [ 0.49346548 -0.13459923 -0.18288507]
 [-0.33173599  0.28654646 -0.01907949]
 [ 0.05616701 -0.03804221 -0.01538686]
 [-0.18307265  0.25097347 -0.10202013]
 [ 0.360051   -0.47401903  0.10542538]
 [ 0.00763673 -0.01309217 -0.00070342]
 [-0.31279462  0.090302    0.03736081]
 [-0.29216397 -0.17286907  0.47436514]
 [ 0.60016647 -0.24529557 -0.26768179]
 [ 0.11925715 -0.07829817 -0.13189722]
 [-0.21064297  0.14394578 -0.12147001]
 [-0.19926883  0.36865518 -0.29633267]
 [-0.35225891  0.22157712  0.45866975]
 [ 0.03175326 -0.18752885  0.05666527]
 [-0.54392451  0.31299127  0.52150181]
 [-0.33238169  0.03008711  0.29936022]
 [-0.14421883 -0.47058757  0.24395825]
 [-0.08363452 -0.60870994  0.68248204]
 [-0.18336155  0.15818716  0.13175612]
 [ 0.05981319 -0.562999    0.45415137]]


In [7]:
assert_near_equal(p.get_val('cross_prod_comp.torque'), np.array(expected_T), tolerance=1e-8)
assert_near_equal(p.get_val('cross_prod_comp.L'), np.array(expected_L), tolerance=1e-8)

0.0